In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D, LeakyReLU
from tensorflow.keras.layers import  Dropout, Activation
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model, to_categorical
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle
from sklearn.preprocessing import MinMaxScaler


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def dataload():
    img = np.load("imgs.npy")
    mask = np.load("masks.npy")

    img = np.expand_dims(img, axis=-1)
    mask = np.expand_dims(mask, axis=-1) 
#     mask = to_categorical(mask) 
    
    return img, mask 

def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = K.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

# def minmaxScaler(data):

def dice_coef(y_true, y_pred, smooth=1):
    y_pred = K.argmax(y_pred, axis=-1)
#     y_true = y_true[:,:,:,0]

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_true_f = K.cast(y_true_f, 'float32')
    y_pred_f = K.cast(y_pred_f, 'float32')

    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#     y_pred_c = K.cast(y_pred_f > 0, 'float32')
#     y_true_c = K.cast(y_true_f > 0, 'float32')
#     intersection = K.sum(K.cast(y_pred_f == y_true_f, 'float32'))
    
#     return (2. * intersection + smooth) / (K.sum(y_true_c) + K.sum(y_pred_c) + smooth)

def dice_coef_multilabel(y_true, y_pred, numLabels=3):
    dice = 0
    
    for index in range(numLabels):
        if index == 0: continue
        y_true_i = K.cast(K.equal(y_true, index), 'float32')
        dice += dice_coef(y_true_i, y_pred)
        
    return dice / (numLabels - 1) # taking average

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_pred = K.argmax(y_pred, axis=-1)
    y_true = y_true[:,:,:,1]
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_true_f = K.cast(y_true_f, 'float32')
    y_pred_f = K.cast(y_pred_f, 'float32')
    
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return tf.math.exp(1  - score) - 1.0
    # return 1. - score

def bce_dice_loss(y_true, y_pred):
    return categorical_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def build_unet(sz=(512,512,1)):
    x = Input(sz)
    inputs = x
  
    #down sampling 
    f = 8
    layers = []
  
    for i in range(0, 6):
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        layers.append(x)
        x = MaxPooling2D() (x)
        f = f*2
        ff2 = 64 
    
    #bottleneck 
    j = len(layers) - 1
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
    x = Concatenate(axis=3)([x, layers[j]])
    j = j -1 
  
    #upsampling 
    for i in range(0, 5):
        ff2 = ff2//2
        f = f // 2 
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        x = Conv2D(f, 3, activation='relu', padding='same') (x)
        x = Conv2DTranspose(ff2, 2, strides=(2, 2), padding='same') (x)
        x = Concatenate(axis=3)([x, layers[j]])
        j = j -1 
    
  
    #classification 
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    x = Conv2D(f, 3, activation='relu', padding='same') (x)
    outputs = Conv2D(3, 1, activation='softmax') (x)
    
    #model creation 
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = [dice_coef_multilabel])
  
    return model


1 Physical GPUs, 1 Logical GPUs


In [2]:
if __name__ == '__main__':
    import os
    os.environ["CUDA_VISIBLE_DEVICES"]="0"

    source, target = dataload()
    print(source.shape, target.shape)
    
    model = build_unet()
    model.summary()

    model.fit(source, target, validation_split=0.1, epochs=10, batch_size=16)

(3890, 512, 512, 1) (3890, 512, 512, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 8)  80          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 8)  584         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 8)  0           conv2d_1[0][0]                   
______________________________________________________

Epoch 1/10
219/219 [==============================] - 55s 253ms/step - loss: 0.2774 - dice_coef_multilabel: 0.0321 - val_loss: 0.0798 - val_dice_coef_multilabel: 0.0443
Epoch 2/10
219/219 [==============================] - 52s 236ms/step - loss: 0.0428 - dice_coef_multilabel: 0.4012 - val_loss: 0.0559 - val_dice_coef_multilabel: 0.3184
Epoch 3/10
219/219 [==============================] - 51s 233ms/step - loss: 0.0308 - dice_coef_multilabel: 0.6765 - val_loss: 0.0486 - val_dice_coef_multilabel: 0.5415
Epoch 4/10
219/219 [==============================] - 51s 233ms/step - loss: 0.0233 - dice_coef_multilabel: 0.6908 - val_loss: 0.0336 - val_dice_coef_multilabel: 0.6336
Epoch 5/10
219/219 [==============================] - 52s 236ms/step - loss: 0.0153 - dice_coef_multilabel: 0.7101 - val_loss: 0.0236 - val_dice_coef_multilabel: 0.6408
Epoch 6/10
219/219 [==============================] - 51s 233ms/step - loss: 0.0096 - dice_coef_multilabel: 0.7194 - val_loss: 0.0234 - val_dice_coef_multi